In [20]:
!wget https://raw.githubusercontent.com/Dewjy02/HPC-ASSIGNMENT/252a52f02a4d76355e5e4c1fe910c35091e6cf31/matrixMultiply/MatData.txt
!wget https://raw.githubusercontent.com/Dewjy02/HPC-ASSIGNMENT/252a52f02a4d76355e5e4c1fe910c35091e6cf31/matrixMultiply/mtx-file-handler.c

--2025-10-22 12:32:38--  https://raw.githubusercontent.com/Dewjy02/HPC-ASSIGNMENT/252a52f02a4d76355e5e4c1fe910c35091e6cf31/matrixMultiply/MatData.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14116 (14K) [text/plain]
Saving to: ‘MatData.txt’

MatData.txt         100%[===================>]  13.79K  --.-KB/s    in 0.001s  

2025-10-22 12:32:38 (24.2 MB/s) - ‘MatData.txt’ saved [14116/14116]

--2025-10-22 12:32:38--  https://raw.githubusercontent.com/Dewjy02/HPC-ASSIGNMENT/252a52f02a4d76355e5e4c1fe910c35091e6cf31/matrixMultiply/mtx-file-handler.c
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443..

In [21]:
%%writefile mtx-multiplication.c
#include <stdio.h>
#include <stdlib.h>
#include <omp.h>
#include "mtx-file-handler.c"

Matrix *multiplyMatrices(Matrix *A, Matrix *B, int threads) {
    if (A->cols != B->rows) {
        printf("Can't multiply (%dx%d) and (%dx%d)\n",
               A->rows, A->cols, B->rows, B->cols);
        return NULL;
    }

    Matrix *result = allocateMatrix(A->rows, B->cols);
    int max_dim = (A->rows > B->cols) ? A->rows : B->cols;
    if (threads > max_dim) threads = max_dim;
    omp_set_num_threads(threads);

    #pragma omp parallel for collapse(2)
    for (int i = 0; i < A->rows; i++) {
        for (int j = 0; j < B->cols; j++) {
            double sum = 0.0;
            for (int k = 0; k < A->cols; k++) {
                sum += A->data[i][k] * B->data[k][j];
            }
            result->data[i][j] = sum;
        }
    }
    return result;
}

int main(int argc, char *argv[]) {
    if (argc < 3) {
        printf("Usage: %s <input_file> <num_threads>\n", argv[0]);
        return 1;
    }

    const char *filename = argv[1];
    int threads = atoi(argv[2]);

    Matrix **matrices = NULL;
    int count = readMatricesFromFile(filename, &matrices);

    if (count < 2) {
        printf("Not enough matrices in file.\n");
        return 1;
    }

    FILE *out = fopen("MatDataResults.txt", "w");
    if (!out) {
        perror("Error creating output file");
        return 1;
    }

    for (int i = 0; i < count - 1; i += 2) {
        printf("Multiplying Matrix %d and %d...\n", i + 1, i + 2);
        Matrix *result = multiplyMatrices(matrices[i], matrices[i + 1], threads);
        if (result) {
            writeMatrixToFile(out, result);
            fprintf(out, "\n");
            freeMatrix(result);
        }
    }

    fclose(out);
    for (int i = 0; i < count; i++) freeMatrix(matrices[i]);
    free(matrices);

    printf("Matrix multiplication completed. Results saved to MatDataResults.txt\n");
    return 0;
}


Overwriting mtx-multiplication.c


In [22]:
%%bash
gcc -fopenmp /content/mtx-multiplication.c -o matmul
./matmul /content/MatData.txt 8

Multiplying Matrix 1 and 2...
Multiplying Matrix 3 and 4...
Multiplying Matrix 5 and 6...
Multiplying Matrix 7 and 8...
Multiplying Matrix 9 and 10...
Multiplying Matrix 11 and 12...
Multiplying Matrix 13 and 14...
Multiplying Matrix 15 and 16...
Multiplying Matrix 17 and 18...
Multiplying Matrix 19 and 20...
Matrix multiplication completed. Results saved to MatDataResults.txt
